In [649]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
%matplotlib inline

In [650]:
#Dataset parçalarının birleştirilmesi
df = pd.read_csv("dataset1.csv")
df2 = pd.read_csv("dataset2.csv")
df = pd.concat([df, df2], axis=0)
df2 = pd.read_csv("dataset3.csv")
df = pd.concat([df, df2], axis=0)
df2 = pd.read_csv("dataset4.csv")
df = pd.concat([df, df2], axis=0)
print(df.shape)

#Datasette Belirtilmemiş değerine sahip olmayan satırlara filtreleme.
df = df[~df.isin(['Belirtilmemiş']).any(axis=1)]
#dataframe olarak alırken Unnamed: 0 şeklinde indis eklenmiş onu dropluyorum.
df.drop("Unnamed: 0",axis="columns",inplace=True)

#Saçma verileri temizlemek ve kategorileri işlemek için özgün verileri yazdırma
def print_unique_v(df):
    for column in df:
        print(f'{column} : {df[column].unique()}')
print_unique_v(df)

(6265, 21)
<bound method Series.unique of 0       False
1       False
2       False
4       False
5       False
        ...  
1046    False
1048    False
1049    False
1050    False
1052    False
Length: 5062, dtype: bool>
Bellek Hızı : ['3200 MHz' '2933 MHz' '2666 MHz' '4800 MHz' '2400 MHz' '2133 MHz'
 '5200 MHz' '4266 MHz' '1066 MHz' '1600 MHz' '4267 MHz' '3733 MHz'
 '6400 MHz']
Cihaz Ağırlığı : ['2 - 4 kg' '1 - 2 kg' '2 kg ve Altı']
Dokunmatik Ekran : ['Yok' 'Var']
Ekran Boyutu : ['15,6 inç' '14 inç' '16,1 inç' '14,1 inç' '17,3 inç' '18 inç' '13,4 inç'
 '13,3 inç' '17 inç' '16 inç' '13 inç']
Ekran Kartı Bellek Tipi : ['GDDR6' 'GDDR5' 'Onboard' 'GDDR4' 'DDR5' 'GDDR5X']
Ekran Kartı Hafızası : ['4 GB' '2 GB' 'Paylaşımlı' '6 GB' '8 GB' '16 GB' '1 GB' '12 GB' '3 GB']
Ekran Kartı Tipi : ['Harici Ekran Kartı' 'Dahili Ekran Kartı'
 'Yüksek Seviye Harici Ekran Kartı']
Ekran Kartı : ['Nvidia GeForce GTX1650' 'Nvidia GeForce MX350' 'Intel UHD Graphics'
 'AMD Radeon RX6500M' 'Intel Irıs Graphic

In [651]:
df.sample(5)

,Bellek Hızı,Cihaz Ağırlığı,Dokunmatik Ekran,Ekran Boyutu,Ekran Kartı Bellek Tipi,Ekran Kartı Hafızası,Ekran Kartı Tipi,Ekran Kartı,Ekran Panel Tipi,Harddisk Kapasitesi,İşlemci Nesli,İşlemci Tipi,İşlemci,İşletim Sistemi,Kart Okuyucu,Maksimum İşlemci Hızı,Max Ekran Çözünürlüğü,Ram (Sistem Belleği),Ram Tipi,Fiyat
1423,3200 MHz,2 - 4 kg,Yok,"15,6 inç",GDDR6,4 GB,Harici Ekran Kartı,Nvidia GeForce GTX1650,IPS,Yok,11.Nesil,Intel Core i5,11320H,Yok (Free Dos),Yok,"4,5 GHz",1920 x 1080,8 GB,DDR4,16.399
511,2666 MHz,1 - 2 kg,Yok,"15,6 inç",GDDR5,2 GB,Harici Ekran Kartı,Nvidia GeForce MX330,TN,Yok,10. Nesil,Intel Core i5,10210U,Windows 10 Home,Var,"4,2 GHz",1920 x 1080,8 GB,DDR4,16.349
1191,3200 MHz,1 - 2 kg,Yok,"15,6 inç",GDDR5,2 GB,Harici Ekran Kartı,Nvidia GeForce MX350,LED,Yok,11.Nesil,Intel Core i5,1135G7,Yok (Free Dos),Yok,"4,2 GHz",1920 x 1080,8 GB,DDR4,9.499
381,3200 MHz,1 - 2 kg,Yok,14 inç,GDDR6,2 GB,Harici Ekran Kartı,Nvidia GeForce MX570,IPS,Yok,12.Nesil,Intel Core i7,1255U,Windows 11 Pro,Var,"4,7 GHz",1920 x 1080,64 GB,DDR4,35.444
1474,3200 MHz,2 - 4 kg,Yok,"15,6 inç",GDDR6,4 GB,Harici Ekran Kartı,Nvidia GeForce RTX 3050,LED,Yok,12.Nesil,Intel Core i7,12700H,Yok (Free Dos),Yok,"4,7 GHz",1920 x 1080,16 GB,DDR4,19.998


In [652]:
df.dtypes


Bellek Hızı                 object
Cihaz Ağırlığı              object
Dokunmatik Ekran            object
Ekran Boyutu                object
Ekran Kartı Bellek Tipi     object
Ekran Kartı Hafızası        object
Ekran Kartı Tipi            object
Ekran Kartı                 object
Ekran Panel Tipi            object
Harddisk Kapasitesi         object
İşlemci Nesli               object
İşlemci Tipi                object
İşlemci                     object
İşletim Sistemi             object
Kart Okuyucu                object
Maksimum İşlemci Hızı       object
Max Ekran Çözünürlüğü       object
Ram (Sistem Belleği)        object
Ram Tipi                    object
Fiyat                      float64
dtype: object

In [653]:
#Feature Processing
min_max_scaler = MinMaxScaler()
label_encoder = LabelEncoder()
ordinal_encoder = OrdinalEncoder()

df['Bellek Hızı'] = df['Bellek Hızı'].str.split(' ').str[0].astype(int)
df['Bellek Hızı'] = min_max_scaler.fit_transform(df[['Bellek Hızı']])

df['Cihaz Ağırlığı'].replace( {'1 - 2 kg' : 0, '2 kg ve Altı' : 0, '2 - 4 kg' : 1}, inplace=True)

df['Dokunmatik Ekran'].replace( {'Yok' : 0, 'Var' : 1 }, inplace=True)

df['Ekran Boyutu'] = df['Ekran Boyutu'].str.split(' ').str[0]
df['Ekran Boyutu'] = df['Ekran Boyutu'].str.replace(",",".").astype(float)

df['Ekran Kartı Bellek Tipi'] = label_encoder.fit_transform(df['Ekran Kartı Bellek Tipi'])

#Paylaşımlı verisinden dolayı ordinal encoding tercih ettim. (İdeal değil)
cat_order=['16 GB','12 GB','8 GB','6 GB','4 GB','3 GB','2 GB','Paylaşımlı','1 GB']
ordinal_encoder= OrdinalEncoder(categories=[cat_order])
df['Ekran Kartı Hafızası'] = ordinal_encoder.fit_transform(df[['Ekran Kartı Hafızası']])

df = pd.get_dummies(data=df, columns=['Ekran Kartı Tipi'])

#Çok fazla farklı ekran kartı olduğu için label_encoder kullandım (İdeal değil.)
df['Ekran Kartı'] = label_encoder.fit_transform(df['Ekran Kartı'])

df['Ekran Panel Tipi'] = label_encoder.fit_transform(df['Ekran Panel Tipi'])

cat_order= ['Yok','128 GB','256 GB','500 GB','512 GB','1 TB','2 TB']
ordinal_encoder= OrdinalEncoder(categories=[cat_order])
df['Harddisk Kapasitesi'] = ordinal_encoder.fit_transform(df[['Harddisk Kapasitesi']])


df.drop(df[(df['İşlemci Nesli'] == 'Intel Core i7-1065G7') | (df['İşlemci Nesli'] == 'İntel N Serisi') ].index, axis='rows', inplace=True)
#Burada nesli belli olmayan bilgisayarlardan dolayı label_encoder kullandım (İdeal değil.)
df['İşlemci Nesli'] = label_encoder.fit_transform(df['İşlemci Nesli'])


#Bazı hatalı verileri temizliyorum.
df.drop(df[ (df['İşlemci Tipi']  == '1235U'   )   | (df['İşlemci Tipi'] == '1255U'    )   |  (df['İşlemci Tipi'] == '12500H'   )   |  (df['İşlemci Tipi']  == '11400H'   )   |  (df['İşlemci Tipi']  == 'i5-11300H'   )   |  (df['İşlemci Tipi']  == '5500U'   )   | (df['İşlemci Tipi'] == '12900H')         ].index , axis='rows', inplace=True)
df.drop(df[ (df['İşlemci'] == '10. Nesil Intel® Core™ i3-10110U işlemci') | (df['İşlemci'] == 'AMD Ryzen 5') ].index , axis='rows',inplace=True)

#İşlemci tipi kategorisini işlemci kategorisiyle birleştiriyorum 
df['İşlemci'] = df['İşlemci Tipi'].str.cat(df['İşlemci'], sep=' ')
df.drop('İşlemci Tipi' , axis='columns' ,inplace=True)
df['İşlemci'] = label_encoder.fit_transform(df['İşlemci'])

df['İşletim Sistemi'] = label_encoder.fit_transform(df['İşletim Sistemi'])

# Minimize etmek için 0(Yok) ile 1(Var) şeklinde ayırdım.
df['Kart Okuyucu'].replace({'MicroSD Card Reader' : 1,'MicroSD ':1, 'Card Reader':1,'MicroSD  Card Reader':1, 'SD Express':1,'Var':1,'Yok':0 }, inplace=True)

#İşlemci hızı olmayanları dropluyorum. Aynı zamanda GHz yazısını silip float olarak değeri tutuyorum.
df.drop(df[ df['Maksimum İşlemci Hızı'] == 'Yok'].index, axis='rows',inplace=True)
df['Maksimum İşlemci Hızı']= df['Maksimum İşlemci Hızı'].str.split(' ').str[0]
df['Maksimum İşlemci Hızı'] = df['Maksimum İşlemci Hızı'].str.replace(",",".").astype(float)


df.drop(df[ (df['Max Ekran Çözünürlüğü'] == '1933 x 1080') | (df['Max Ekran Çözünürlüğü'] == '1921 x 1080') ].index, axis='rows', inplace=True)
df['Max Ekran Çözünürlüğü'] = label_encoder.fit_transform(df['Max Ekran Çözünürlüğü'])

df['Ram (Sistem Belleği)']= df['Ram (Sistem Belleği)'].str.split(' ').str[0].astype(int)

df['Ram Tipi'] = label_encoder.fit_transform(df['Ram Tipi'])


df['Fiyat']=(df['Fiyat']*1000).astype(int)

print_unique_v(df)
df.dtypes

Bellek Hızı : [0.40007499 0.35001875 0.2999625  0.7000375  0.25009374 0.2000375
 0.77502812 0.59992501 0.         0.60011249 0.5        1.        ]
Cihaz Ağırlığı : [1 0]
Dokunmatik Ekran : [0 1]
Ekran Boyutu : [15.6 14.  16.1 17.3 13.4 13.3 17.  16.  13.  18. ]
Ekran Kartı Bellek Tipi : [4 2 5 1 0]
Ekran Kartı Hafızası : [4. 6. 7. 3. 2. 0. 1.]
Ekran Kartı : [25 29 21  7 20 51 36 28 37  0 35 30 39 38 18 50 42 19 41 40 31 12 46 10
 22 13 32  4 17  2 26  8  1  9  5 44 11 27  3 43 45 47 33 49 48]
Ekran Panel Tipi : [6 2 1 0 7 3 4 5]
Harddisk Kapasitesi : [2. 0. 5. 4. 3. 1. 6.]
İşlemci Nesli : [ 8  2  1  3  6  0 11 10  9  4  5  7]
İşlemci : [ 6 38 31 60 30 26 45  0 39 24 21 37 29 63 11 52 44 12 66 62  8 54 51 47
 17  5 55 15 41 67  2 18 14 25 42  4 16 69  7 59 34 22 20 36 49 27 32 43
 35 53 33 73 61 50 72 71 57  3  9 68 23  1 65 46 13 70 28 56 58 48 64 10
 40]
İşletim Sistemi : [8 3 1 5 2 7 0 6 4]
Kart Okuyucu : [1 0]
Maksimum İşlemci Hızı : [4.2 3.6 4.7 4.5 4.1 3.5 3.4 2.6 4.3 4.6 4.4 5. 

Bellek Hızı                                          float64
Cihaz Ağırlığı                                         int64
Dokunmatik Ekran                                       int64
Ekran Boyutu                                         float64
Ekran Kartı Bellek Tipi                                int32
Ekran Kartı Hafızası                                 float64
Ekran Kartı                                            int32
Ekran Panel Tipi                                       int32
Harddisk Kapasitesi                                  float64
İşlemci Nesli                                          int32
İşlemci                                                int32
İşletim Sistemi                                        int32
Kart Okuyucu                                           int64
Maksimum İşlemci Hızı                                float64
Max Ekran Çözünürlüğü                                  int32
Ram (Sistem Belleği)                                   int32
Ram Tipi                

In [654]:
import sys
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

X=df.drop("Fiyat", axis="columns")
y=df["Fiyat"]

iteration=25 # Kaç farklı defa veriseti regresyonu için iterasyon değeri
table=[] # İşlem sonuçlarının tutulduğu 3-d dizi  

for i in range(iteration):
    #Verisetinin 10 katlı çapraz geçerlenmesi
    X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.1)
    
    #Üstteki blokta genel olarak yapılmış olsa da bura da da scaling uyguluyorum.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    
    
    
    #Farklı regresyon modellerinin tutulduğu değişken
    regression_models = [LinearRegression(),
                        linear_model.Lasso(alpha=0.5, max_iter=10000),
                        linear_model.ElasticNet(alpha=0.01, max_iter=10000),
                        linear_model.SGDRegressor(max_iter=10000),
                        linear_model.RANSACRegressor(),
                        SVR(kernel="rbf", C=100000, gamma=0.1, epsilon=0.1)] 
    
    #Bu kısımda her regresyon modeli için Root Mean Square Error ile R-Square sonucu hesaplanıp table'a ekleniyor
    value_mat=[]
    for model in regression_models:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r_squared = model.score(X_test_scaled, y_test)
        value_mat.append([rmse,r_squared])
    table.append(value_mat) 
 
#Burası RMSE ve R-Square sonuçlarının minimum, average, maximum değerlerinin hesaplandığı yer   
matrix=[([0]*2) for i in range(6)]
max_values=[([0]*2) for i in range(6)]
min_values=[([sys.maxsize]*2) for i in range(6)]
for i in range(len(table)):
    for j in range(len(table[i])):  
        for k in range(len(table[i][j])):
            matrix[j][k]+=table[i][j][k]
        if(max_values[j][1]<table[i][j][1]):
            max_values[j]=table[i][j]
        if(min_values[j][1]>table[i][j][1]):
            min_values[j]=table[i][j]
            

In [655]:
np.set_printoptions(suppress=True)
nmatrix= np.array(matrix)/iteration
from tabulate import tabulate

#Verilerin tablolaştırılması
data=[["LinearRegression",max_values[0][1],nmatrix[0][1],min_values[0][1],max_values[0][0],nmatrix[0][0],min_values[0][0]],
      ["Lasso",max_values[1][1],nmatrix[1][1],min_values[1][1],max_values[1][0],nmatrix[1][0],min_values[1][0]],
      ["ElasticNet",max_values[2][1],nmatrix[2][1],min_values[2][1],max_values[2][0],nmatrix[2][0],min_values[2][0]],
      ["SGD",max_values[3][1],nmatrix[3][1],min_values[3][1],max_values[3][0],nmatrix[3][0],min_values[3][0]],
      ["RANSAC",max_values[4][1],nmatrix[4][1],min_values[4][1],max_values[4][0],nmatrix[4][0],min_values[4][0]],
      ["SVR",max_values[5][1],nmatrix[5][1],min_values[5][1],max_values[5][0],nmatrix[5][0],min_values[5][0]]]
print(tabulate(data, headers=["Algorithms/Results", "Max R_square", "Average R_square", "Min R_square","Lowest RMSE","Average RMSE", "Highest RMSE"],tablefmt="fancy_grid"))


╒══════════════════════╤════════════════╤════════════════════╤════════════════╤═══════════════╤════════════════╤════════════════╕
│ Algorithms/Results   │   Max R_square │   Average R_square │   Min R_square │   Lowest RMSE │   Average RMSE │   Highest RMSE │
╞══════════════════════╪════════════════╪════════════════════╪════════════════╪═══════════════╪════════════════╪════════════════╡
│ LinearRegression     │       0.869208 │           0.801375 │       0.728016 │       3018.27 │        3857.57 │        4534.28 │
├──────────────────────┼────────────────┼────────────────────┼────────────────┼───────────────┼────────────────┼────────────────┤
│ Lasso                │       0.869214 │           0.801465 │       0.72795  │       3018.21 │        3857.15 │        4534.83 │
├──────────────────────┼────────────────┼────────────────────┼────────────────┼───────────────┼────────────────┼────────────────┤
│ ElasticNet           │       0.870139 │           0.800208 │       0.723181 │       3007

In [656]:
from scipy.stats import ttest_ind

#Algoritma sonuçlarının table' dan daha düzgün bir şekilde ayrıştırılması (iter,alg,sonuç) iken (alg,iter,sonuç) haline getirdim. (Bunu baştan yapıp üst blokta aynı işlemleri yapmak mantıklı olurdu.)
alg_scores=[]
for j in range(len(table[i])):
    alg=[]
    for i in range(len(table)):
        alg.append(table[i][j][1])
    alg_scores.append(alg)
    
#2 algoritma arasında T-test için fonksiyon tanımlaması
def t_test(i,j):
    t_statistic, p_value = ttest_ind(alg_scores[i], alg_scores[j])
    alpha = 0.05  # Anlamlılık düzeyi
    if p_value < alpha:
        anlam="Var"
    else:
        anlam="Yok"
    return anlam
data=[["LinearRegression"," ",t_test(0,1),t_test(0,2),t_test(0,3),t_test(0,4),t_test(0,5)],
      ["Lasso",t_test(1,0)," ",t_test(1,2),t_test(1,3),t_test(1,4),t_test(1,5)],
      ["ElasticNet",t_test(2,0),t_test(2,1)," ",t_test(2,3),t_test(2,4),t_test(2,5)],
      ["SGD",t_test(3,0),t_test(3,1),t_test(3,2)," ",t_test(3,4),t_test(3,5)],
      ["RANSAC",t_test(4,0),t_test(4,1),t_test(4,2),t_test(4,3)," ",t_test(4,5)],
      ["SVR",t_test(5,0),t_test(5,1),t_test(5,2),t_test(5,3),t_test(5,4)," "]]
print(tabulate(data, headers=["T-test anlamlı fark\n(alpha = 0.05)","LinearRegression", "Lasso", "ElasticNet", "SGD","RANSAC","SVR"],tablefmt="fancy_grid"))




╒═══════════════════════╤════════════════════╤═════════╤══════════════╤═══════╤══════════╤═══════╕
│ T-test anlamlı fark   │ LinearRegression   │ Lasso   │ ElasticNet   │ SGD   │ RANSAC   │ SVR   │
│ (alpha = 0.05)        │                    │         │              │       │          │       │
╞═══════════════════════╪════════════════════╪═════════╪══════════════╪═══════╪══════════╪═══════╡
│ LinearRegression      │                    │ Yok     │ Yok          │ Yok   │ Var      │ Var   │
├───────────────────────┼────────────────────┼─────────┼──────────────┼───────┼──────────┼───────┤
│ Lasso                 │ Yok                │         │ Yok          │ Yok   │ Var      │ Var   │
├───────────────────────┼────────────────────┼─────────┼──────────────┼───────┼──────────┼───────┤
│ ElasticNet            │ Yok                │ Yok     │              │ Yok   │ Var      │ Var   │
├───────────────────────┼────────────────────┼─────────┼──────────────┼───────┼──────────┼───────┤
│ SGD     